In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

In [2]:
# Baca data
file_path = 'inflasibulanfix.csv'
df = pd.read_csv(file_path)

In [3]:
# Ambil kolom yang relevan
df = df[['City', 'Month', 'Year', 'Inflation']]

In [4]:
# Ganti koma dengan titik pada kolom 'Inflation'
df['Inflation'] = df['Inflation'].str.replace(',', '.').astype(float)

In [5]:
# Urutkan data
df = df.sort_values(by=['City', 'Year', 'Month'])

In [6]:
# Scaling data inflasi
scaler = MinMaxScaler()
df['Inflation'] = scaler.fit_transform(df['Inflation'].values.reshape(-1, 1))

In [7]:
# Pisahkan data menjadi seri waktu untuk setiap kota
city_data = {}
for city in df['City'].unique():
    city_data[city] = df[df['City'] == city]['Inflation'].values

In [8]:
# Fungsi untuk membuat sekuens waktu
def create_time_series(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        a = data[i:(i + time_steps)]
        X.append(a)
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

In [9]:
# Hyperparameter
time_steps = 12  # Sesuaikan dengan panjang sekuens waktu yang ingin Anda gunakan
n_features = 1  # Jumlah fitur (hanya inflasi)

In [11]:
# Membuat dan melatih model LSTM untuk setiap kota
models = {}
for city, data in city_data.items():
    X, y = create_time_series(data, time_steps)
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(time_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=50, verbose=1)
    
    models[city] = model

Epoch 1/50
6/6 [==============================] - 3s 6ms/step - loss: 0.1205
Epoch 2/50
6/6 [==============================] - 0s 7ms/step - loss: 0.0670
Epoch 3/50
6/6 [==============================] - 0s 6ms/step - loss: 0.0281
Epoch 4/50
6/6 [==============================] - 0s 7ms/step - loss: 0.0050
Epoch 5/50
6/6 [==============================] - 0s 9ms/step - loss: 0.0029
Epoch 6/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0044
Epoch 7/50
6/6 [==============================] - 0s 6ms/step - loss: 0.0010
Epoch 8/50
6/6 [==============================] - 0s 6ms/step - loss: 0.0016
Epoch 9/50
6/6 [==============================] - 0s 6ms/step - loss: 0.0013
Epoch 10/50
6/6 [==============================] - 0s 6ms/step - loss: 8.9750e-04
Epoch 11/50
6/6 [==============================] - 0s 6ms/step - loss: 9.5530e-04
Epoch 12/50
6/6 [==============================] - 0s 6ms/step - loss: 9.0793e-04
Epoch 13/50
6/6 [==============================] - 0s 6ms/ste

In [12]:
# Contoh prediksi untuk suatu kota
sample_city = 'KOTA YOGYAKARTA'
input_data = scaler.transform(city_data[sample_city][-time_steps:].reshape(-1, 1))
input_data = input_data.reshape((1, time_steps, n_features))
predicted_inflation = models[sample_city].predict(input_data)

1/1 [==============================] - 0s 325ms/step


In [13]:
# Inverse transform hasil prediksi untuk mendapatkan nilai sebenarnya
predicted_inflation_actual = scaler.inverse_transform(predicted_inflation.reshape(-1, 1))

In [14]:
print(f'Predicted Inflation for {sample_city}: {predicted_inflation_actual[0][0]}')

Predicted Inflation for KOTA YOGYAKARTA: 0.3421638309955597
